In [1]:
import sys
sys.path.append('/lmh_data/work/sclab/train/')

import numpy as np
import pandas as pd
import torch
import torch.utils.data as data
from torch.autograd import Variable
from captum.attr import IntegratedGradients

from dataset import Dataset
from net import define_network

In [2]:
torch.manual_seed(123)
np.random.seed(123)

In [3]:
def evaluate(eval_file, model_file, gene_name):
    eval_set = Dataset(eval_file, gene_name, is_train=False)
    data_loader = data.DataLoader(eval_set, batch_size=1, shuffle=False)

    input_size, output_size = tuple(eval_set[0][0].shape), 45
    patch_size = tuple([int(i / 8) for i in input_size])
    model = torch.nn.DataParallel(define_network(input_size, patch_size, output_size))
    model.load_state_dict(torch.load(model_file))
    model.cuda()
    model.eval()
    ig = IntegratedGradients(model)

    _datas = np.load(eval_file, allow_pickle=True)
    important_genes = dict(ASC=dict(), Endo=dict(), OPC=dict(), ODC=dict())
    for index, batch in enumerate(data_loader, 1):
        if _datas[index-1]['cell_type'] != 'ASC':
            continue
        _important_genes = important_genes[_datas[index-1]['cell_type']]
        input = Variable(batch[0]).cuda().unsqueeze(1)
#         target = Variable(batch[1]).cuda().unsqueeze(1) / 10
        target = model(input)
        
        attributions = torch.zeros(input.reshape(-1).shape).cuda()
        for i in range(target.shape[-1]):
            attributions += ig.attribute(input, target=(0, i))[0, 0].reshape(-1)
        attributions = attributions.cpu().detach().numpy()
        for i in range(len(attributions)):
            if i not in _important_genes.keys():
                _important_genes[i] = 0
            _important_genes[i] += attributions[i]
        
    return important_genes

important_genes = evaluate(
    '/lmh_data/data/sclab/sclab/AD/eval_dataset.npy',
    '/lmh_data/data/sclab/sclab/AD/tmp/SLC1A3/model_epoch_9.pth', 'SLC1A3')
important_genes

{'ASC': {0: -0.025375110192726424,
  1: -0.0006924183280716534,
  2: 0.0,
  3: -0.01529272086918354,
  4: 0.0,
  5: -0.11163147061597556,
  6: 0.043442979687824845,
  7: -0.004376029362902045,
  8: 0.0,
  9: -0.005570125009398907,
  10: 0.5472732663620263,
  11: -0.018998323881532997,
  12: 0.003621804082285962,
  13: -0.0004080799699295312,
  14: 0.0106238997541368,
  15: -0.0046868224162608385,
  16: 0.0,
  17: 0.0,
  18: 0.0,
  19: -0.07027113472577184,
  20: -0.3810268137603998,
  21: 0.0,
  22: 0.019080360943917185,
  23: 0.0,
  24: 0.0,
  25: 0.0008242675103247166,
  26: 0.17424125972320326,
  27: 0.004385590786114335,
  28: 0.0,
  29: 0.0,
  30: -0.5308616689871997,
  31: -0.11603679601103067,
  32: 0.0,
  33: -0.002343275248676946,
  34: 0.0,
  35: -0.03916354657849297,
  36: 0.0001506898261141032,
  37: -0.0030933681409806013,
  38: 0.0,
  39: -0.16730289376573637,
  40: 0.0,
  41: 0.0,
  42: 0.08166565909050405,
  43: 0.019656765856780112,
  44: 0.13134172017453238,
  45: 0.0

In [4]:
scRNA_head = np.load('/lmh_data/data/sclab/sclab/AD/eval_dataset.npy', allow_pickle=True)[0]['scRNA_head']

def find_special_genes(important_genes, cell_type):
    genes = pd.DataFrame(
        sorted(important_genes[cell_type].items(), key=lambda x: x[1], reverse=True), columns=['index', 'value']
    ).set_index('index')
    genes = genes.sort_values(by=['value'], ascending=False)
    return genes

genes = find_special_genes(important_genes, 'ASC')
genes['name'] = scRNA_head[list(genes.index)]
genes.to_csv('tmp/related_genes_SLC1A3.csv')
genes

,value,name
index,,
3904,30.859935,SLC1A2
16466,19.153897,ZBTB20
7721,17.935119,CTNNA2
12171,16.684254,LRP1B
5819,14.857453,MAGI2
...,...,...
16478,-10.070380,GPHN
1069,-12.409622,TNIK
1806,-13.805932,NPAS3
